### Resumo das Rotinas de Manutenção:
- **Vacuum**: Remove arquivos antigos para liberar espaço.
- **Optimize**: Combina arquivos pequenos para melhorar a performance de leitura.
- **Z-Ordering**: Organiza os dados para melhorar o desempenho em consultas filtradas.
- **Update/Delete**: Permite operações eficientes de modificação de dados.
- **History/Time Travel**: Audita e acessa versões anteriores dos dados.
- **Compaction**: Agrupa arquivos pequenos para melhorar a eficiência de leitura.
- Essas práticas de manutenção são fundamentais para gerenciar com eficiência um Delta Lake, mantendo a performance e a integridade dos dados.

Manter um** Delta Lake** bem gerenciado é fundamental para garantir a performance, a integridade dos dados e o uso eficiente de recursos. Aqui estão as principais rotinas de manutenção do Delta Lake, quando, como e por que usá-las:

### 1. Vacuum
**Quando usar**: Para remover arquivos antigos que não são mais necessários, como aqueles gerados por operações de update, merge ou delete.

**Por que usar**: O Delta Lake mantém versões antigas de dados (histórico) para fornecer recursos como time travel e rollback. Com o tempo, esses arquivos antigos podem consumir muito espaço em disco. O vacuum remove esses arquivos, liberando espaço.

**Recomendação**: Evite configurar o período de retenção abaixo de 7 dias sem considerar as implicações no time travel. O padrão de 7 dias é seguro para manter a possibilidade de recuperação de dados e, ao mesmo tempo, limpar arquivos obsoletos.

In [0]:
from delta.tables import DeltaTable

# Define os caminhos de armazenamento no Data Lake
gold_path = "/mnt/lhdw/gold/vendas_delta/fato_vendas"

# Disable retention duration check
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false")
# Executa vacuum para remover arquivos não utilizados com mais de 7 dias
delta_table = DeltaTable.forPath(spark, gold_path)
delta_table.vacuum(7)

Out[1]: DataFrame[]

**Importando DeltaTable**
- DeltaTable é uma API do Delta Lake que permite manipular tabelas Delta diretamente, possibilitando operações como vacuum, merge, update e outras.

**Definindo o Caminho da Tabela Delta**
- Esse é o caminho onde a tabela fato_vendas está armazenada no formato Delta. O caminho leva até a camada Gold do seu data lake.

**Desativando Verificação de Retenção Padrão**
- Por padrão, o Delta Lake exige que o vacuum seja executado com um período de retenção mínimo de 7 dias. Para evitar essa restrição, essa configuração desativa a verificação de duração mínima de retenção.
  - "false": Permite ignorar a verificação e executar o vacuum com menos de 7 dias de retenção.

**Instanciando a Tabela Delta**
- **DeltaTable.forPath()**: Carrega a tabela Delta do caminho especificado (gold_path) para a variável delta_table, permitindo realizar operações como limpeza de arquivos.

**Executando o Vacuum**
- O vacuum **delta_table.vacuum(7)** é usado para remover arquivos antigos ou não referenciados (ou seja, dados que foram deletados ou sobrescritos em versões anteriores).
  -7: Esse valor define que o vacuum irá remover arquivos que não foram usados nos últimos 7 dias.

**Resumo**: Este código executa uma limpeza de arquivos obsoletos para a tabela fato_vendas no caminho definido, removendo dados que não são mais necessários e que estão fora do período de retenção de 7 dias. O objetivo é liberar espaço de armazenamento no sistema de arquivos.

### 2. Optimize
**Quando usar**: Para otimizar o layout dos arquivos armazenados no Delta Lake, especialmente após muitas operações de escrita ou atualização que podem gerar arquivos pequenos.

**Por que usar**: O Delta Lake pode acabar com muitos arquivos pequenos após operações de escrita ou merge. Isso pode prejudicar o desempenho das consultas devido ao overhead de leitura de muitos arquivos. O optimize combina arquivos pequenos em arquivos maiores, melhorando a leitura e o processamento.

**Recomendação**: Use o optimize em intervalos regulares ou após grandes operações de escrita, para garantir que o layout dos dados continue eficiente. Para melhorar ainda mais o desempenho, o optimize pode ser combinado com Z-Ordering.

In [0]:
from delta.tables import *

# Define os caminhos de armazenamento no Data Lake
gold_path = "/mnt/lhdw/gold/vendas_delta/fato_vendas"
# Otimiza a tabela combinando arquivos pequenos em arquivos maiores
delta_table = DeltaTable.forPath(spark, gold_path)
delta_table.optimize().executeCompaction()

Out[2]: DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bi

- **delta_table.optimize()**: Este comando é usado para otimizar a tabela Delta. Ele reorganiza e agrupa pequenos arquivos em arquivos maiores. Isso pode melhorar significativamente o desempenho de leitura, já que o Spark vai processar menos arquivos ao acessar os dados.

- **executeCompaction()**: A compactação efetiva é executada aqui, após a chamada para **optimize()**. A compactação reduz a fragmentação de dados ao combinar arquivos menores em arquivos maiores.

### 3. Z-Ordering
**Quando usar:** Para otimizar as consultas que realizam filtragens frequentes em determinadas colunas, como colunas de data ou chave.

**Por que usar:** O Z-Ordering melhora o desempenho da leitura ao organizar fisicamente os dados em disco com base em uma coluna ou conjunto de colunas, reduzindo o tempo necessário para buscar os registros filtrados.

**Recomendação:** Use o Z-Ordering em colunas que são frequentemente usadas em cláusulas de filtro para melhorar a leitura de dados relacionados. Combine isso com o optimize para ter dados organizados de forma mais eficiente no disco.

In [0]:
from delta.tables import DeltaTable, DeltaOptimizeBuilder
# Executa optimize com Z-Ordering na coluna "DataVenda"

gold_path = "/mnt/lhdw/gold/vendas_delta/fato_vendas"

# Load the Delta table
delta_table = DeltaTable.forPath(spark, gold_path)

# Execute Z-Ordering optimization on the column "DataVenda"
#delta_table.optimize().#executeZOrderBy("DataVenda").#execute()

spark.sql(f"""OPTIMIZE delta.`{gold_path}` ZORDER BY (DataVenda)""")

Out[3]: DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bi

- **OPTIMIZE**: 
  - O comando **OPTIMIZE** é uma operação de otimização nas tabelas Delta que melhora o desempenho de leitura ao compactar arquivos pequenos em arquivos maiores. A compactação é especialmente útil em tabelas que recebem muitas pequenas inserções ou atualizações.

- **ZORDER**: 
  - O **ZORDER** é uma técnica de ordenação de dados que organiza os arquivos para melhorar a eficiência das consultas que filtram por colunas específicas.
  - **ZORDER BY (DataVenda)**: Neste caso, estamos usando o ZORDER para organizar os dados com base na coluna **DataVenda**. Isso melhora o desempenho das consultas que utilizam filtros ou agregações com base nessa coluna (por exemplo, quando você consulta vendas por data ou em intervalos de datas).

### 4. Update and Delete Operations (UPSERT)
**Quando usar:** Para modificar ou remover dados diretamente em uma tabela Delta, sem precisar sobrescrever a tabela inteira.

**Por que usar:** O Delta Lake permite que você faça operações upsert (combinação de update e insert) e delete, o que é fundamental em pipelines de dados que exigem dados corrigidos, removidos ou atualizados continuamente, como tabelas de fatos ou históricos.

**Recomendação**: Essas operações são úteis para ajustar os dados de maneira eficiente, especialmente quando o volume de dados não é massivo ou quando os dados precisam ser corrigidos frequentemente.

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import lit, max, current_timestamp

# Carregar a tabela Delta
delta_table = DeltaTable.forPath(spark, "/mnt/lhdw/gold/vendas_delta/dim_fabricante")

# Calcular o próximo valor de SK_Fabricante
next_sk = delta_table.toDF().select(max("sk_fabricante")).collect()[0][0] + 1

# Criar uma nova linha para ser inserida
new_row = spark.createDataFrame([
    (8, "Novo Fabricante", next_sk)  # Supondo que 8 seja um IDFabricante de exemplo
], ["IDFabricante", "Fabricante", "sk_fabricante"])

# Adicionar a coluna DataAtualizacao com o tipo correto
new_row = new_row.withColumn("data_atualizacao", current_timestamp())

# Executar a operação de inserção
delta_table.alias("target").merge(
    new_row.alias("source"),
    "target.IDFabricante = source.IDFabricante"
).whenNotMatchedInsertAll().execute()

### Exemplo de Update

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col, current_timestamp

# Exemplo de update
delta_table = DeltaTable.forPath(spark, "/mnt/lhdw/gold/vendas_delta/dim_fabricante")
delta_table.update(
    condition = col("IDFabricante") == 7,  # Condição de update
    set = { 
        "Fabricante": "'VanArsdel Inc.'",
        "data_atualizacao": current_timestamp()
    }  # Atualização de valor
)

### Exemplo de Delete

In [0]:
# Exemplo de delete
delta_table.delete(condition = col("IDFabricante") == 8)

### Exemplo de UPSERT

In [0]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
from pyspark.sql.functions import monotonically_increasing_id, current_timestamp

# Carregue o DataFrame de origem (novos dados)
df_silver = spark.read.format("parquet").load("/mnt/lhdw/silver/vendas/")

#Nome tabela destino
tb_destino = "dim_fabricante"

# Extrair produtos únicos para a dimensão Fabricante    
dim_fabricante_df = df_silver.select(
    "IDFabricante", "Fabricante").dropDuplicates()

# Adicionar chave substituta (surrogate keys)
dim_fabricante_df = dim_fabricante_df.withColumn("sk_fabricante", monotonically_increasing_id()+1)
# Carregue o DataFrame de destino (tabela existente)
df_target = DeltaTable.forPath(spark, "/mnt/lhdw/gold/vendas_delta/dim_fabricante")

# Realize a operação de merge
df_target.alias("target").merge(
    dim_fabricante_df.alias("source"),
    "target.IDFabricante = source.IDFabricante"
).whenMatchedUpdate(set={
    "Fabricante": "source.Fabricante",
    "SK_Fabricante": "source.SK_Fabricante",
    "data_atualizacao": current_timestamp()
}).whenNotMatchedInsert(values={
    "Fabricante": "source.Fabricante",
    "IDFabricante": "source.IDFabricante",
    "SK_Fabricante": "source.SK_Fabricante",
    "data_atualizacao": current_timestamp()
}).execute()

### 5. History e Time Travel
**Quando usar:** Para auditar mudanças na tabela Delta ou para acessar versões anteriores dos dados.

**Por que usar:** O Delta Lake mantém um log de transações que permite rastrear todas as modificações feitas na tabela. Isso é útil para auditoria e recuperação de dados em um ponto anterior no tempo.

**Recomendação:** Use o histórico e o time travel para depurar problemas ou restaurar versões anteriores dos dados quando necessário. No entanto, lembre-se de usar o vacuum para gerenciar a quantidade de histórico mantido.

### Acessando o histórico da tabela:

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.functions import *

# Visualizar o histórico da tabela Delta
history_df = DeltaTable.forPath(spark, "/mnt/lhdw/gold/vendas_delta/dim_fabricante").history()

display(history_df)

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2025-03-14T20:27:02.000+0000,3909164570725886,marlosigordevops@gmail.com,MERGE,"Map(predicate -> [""(IDFabricante#7893 = IDFabricante#7839)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2786782628829924),0314-194801-nfxye9n,4,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 1476, numTargetBytesRemoved -> 1511, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 1, executionTimeMs -> 11564, materializeSourceTimeMs -> 6700, numTargetRowsInserted -> 0, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 2912, numTargetRowsUpdated -> 1, numOutputRows -> 1, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 1, numTargetFilesRemoved -> 1, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 1611)",null,Databricks-Runtime/12.2.x-scala2.12
4,2025-03-14T20:26:37.000+0000,3909164570725886,marlosigordevops@gmail.com,DELETE,"Map(predicate -> [""(IDFabricante#6602 = 8)""])",null,List(2786782628829924),0314-194801-nfxye9n,3,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1518, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2069, numDeletedRows -> 1, scanTimeMs -> 1210, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 857)",null,Databricks-Runtime/12.2.x-scala2.12
3,2025-03-14T20:26:30.000+0000,3909164570725886,marlosigordevops@gmail.com,UPDATE,"Map(predicate -> [""(IDFabricante#6602 = 7)""])",null,List(2786782628829924),0314-194801-nfxye9n,2,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1476, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2684, scanTimeMs -> 1626, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1511, rewriteTimeMs -> 1045)",null,Databricks-Runtime/12.2.x-scala2.12
2,2025-03-14T20:26:18.000+0000,3909164570725886,marlosigordevops@gmail.com,MERGE,"Map(predicate -> [""(cast(IDFabricante#5422 as bigint) = IDFabricante#5641L)""], matchedPredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2786782628829924),0314-194801-nfxye9n,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 1518, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 5649, materializeSourceTimeMs -> 4, numTargetRowsInserted -> 1, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 0, numTargetRowsUpdated -> 0, numOutputRows -> 1, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 1, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 3218)",null,Databricks-Runtime/12.2.x-scala2.12
1,2025-03-14T15:13:01.000+0000,3909164570725886,marlosigordevops@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(2759344008297903),0314-121817-hvvljzxn,0,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1476)",null,Databricks-Runtime/12.2.x-scala2.12
0,2025-03-14T13:25:13.000+0000,3909164570725886,marlosigordevops@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(249561348954126),0314-121817-hvvljzxn,null,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1168)",null,Databricks-Runtime/12.2.x-scala2.12


### Acessar versões antigas (Time Travel):

In [0]:
# Acessar a versão 5 da tabela
df = spark.read.format("delta").option("versionAsOf", 5).load("/mnt/lhdw/gold/vendas_delta/dim_fabricante")
display(df)

IDFabricante,Fabricante,sk_fabricante,data_atualizacao
7,VanArsdel,1,2025-03-14T20:26:50.063+0000


### 6. Restaurar uma versão antiga de tabela Delta

In [0]:
from delta.tables import DeltaTable

# Inicializar DeltaTable
delta_table_path = "dbfs:/mnt/lhdw/gold/vendas_delta/dim_fabricante"
delta_table = DeltaTable.forPath(spark, delta_table_path)

# Restaurar a tabela para a versão 
delta_table.restoreToVersion(2)

# Apresentar a tabela
display(spark.read.format("delta").load(delta_table_path))

IDFabricante,Fabricante,sk_fabricante,data_atualizacao
8,Novo Fabricante,2,2025-03-14T20:26:11.791+0000
7,VanArsdel,1,2025-03-14T15:12:55.113+0000


### 7. Compaction (Compactação)

**Quando usar**: Para agrupar arquivos pequenos resultantes de múltiplas operações de escrita em arquivos maiores, melhorando a performance de leitura.

**Por que usar**: Com o tempo, as operações de escrita podem gerar muitos arquivos pequenos, resultando em um número excessivo de partições pequenas, o que afeta a performance. A compactação agrupa esses arquivos pequenos para melhorar o desempenho de leitura e reduzir o overhead.

**Recomendação:** Execute operações de compactação regularmente ou após grandes operações de escrita para manter o layout dos dados em arquivos otimizados.

In [0]:
# Reparticionando a tabela fato_vendas por ano e mês e salvando o resultado
# Agrupando os arquivos pequenos em arquivos maiores
df = spark.read.format("delta").load("/mnt/lhdw/gold/vendas_delta/fato_vendas")
df.repartition(2).write.format("delta").mode("overwrite").save("dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas_repart")

**1. Particionamento com `repartition`**

O `repartition` é utilizado para aumentar ou reduzir o número de partições de forma uniforme, redistribuindo os dados através de um shuffle. Útil quando você precisa de mais paralelismo.

In [0]:
# Exemplo de criação de um DataFrame 
df_geo = spark.read.format("delta").load("/mnt/lhdw/gold/vendas_delta/dim_geografia")  
# Lendo um arquivo Parquet como exemplo

# Verificar número de partições iniciais
print(f"Número de partições iniciais: {df_geo.rdd.getNumPartitions()}")

# Redefinir para 2 partições usando repartition
df_reparticionado = df_geo.repartition(2)

# Persiste os dados em uma tabela Delta
df_reparticionado.write.format("delta").option("overwriteSchema", "true").mode("overwrite").save("/mnt/lhdw/gold/vendas_delta/geo_repart")

# Verificar número de partições após repartition
print(f"Número de partições após repartition: {df_reparticionado.rdd.getNumPartitions()}")

Número de partições iniciais: 2
Número de partições após repartition: 2


**2. Reparticionamento com Coluna Específica**

Se o dataset contiver uma coluna-chave (como Regiao ou Data), você pode usar o repartition para redistribuir os dados com base em uma coluna específica, o que pode ser útil para garantir que os dados relacionados sejam processados juntos.

In [0]:
df_geo = spark.read.format("delta").load("/mnt/lhdw/gold/vendas_delta/dim_geografia")  
# Reparticionando os dados pela coluna "Regiao"
df_reparticionado_geo = df_geo.repartition(10, "Regiao")


# Persiste os dados em uma tabela Delta
df_reparticionado_geo.write.partitionBy("Regiao").format("delta").option("overwriteSchema", "true").mode("overwrite").save("/mnt/lhdw//gold/vendas_delta/geo_regiao")

# Verificar número de partições após reparticionar pela coluna "Regiao"
print(f"Número de partições após reparticionar pela coluna Regiao: {df_reparticionado_geo.rdd.getNumPartitions()}")

Número de partições após reparticionar pela coluna Regiao: 10


**3. Redução de Partições com `coalesce`**
O `coalesce` é utilizado para reduzir o número de partições sem realizar um shuffle, o que é útil quando se deseja consolidar partições e reduzir o número de tarefas, como ao escrever para o disco.

In [0]:
# Usando coalesce para reduzir as partições para 5
df_coalesced = df_geo.repartition(100).coalesce(5)

# Persiste os dados em uma tabela Delta
df_coalesced.write.format("delta").mode("overwrite").save("/mnt/lhdw/gold/vendas_delta/geo_coaslece")

# Persiste os dados em uma tabela Delta
df_coalesced.write.format("delta").option("overwriteSchema", "true").mode("overwrite").save("/mnt/lhdw/gold/vendas_delta/geo_coalesce")

# Verificar número de partições após o coalesce
print(f"Número de partições após coalesce: {df_coalesced.rdd.getNumPartitions()}")

Número de partições após coalesce: 5


**Resumo das Técnicas:**

- `repartition(n)`: Redistribui os dados uniformemente em n partições. Útil para aumentar o número de partições ou garantir uma melhor distribuição.
- `repartition(col)`: Redistribui os dados com base em uma ou mais colunas, garantindo que valores semelhantes estejam na mesma partição.
- `coalesce(n)`: Reduz o número de partições sem um shuffle, consolidando as partições existentes de forma eficiente.

**Quando Usar:**

- `repartitio`n: Use quando quiser aumentar o número de partições ou redistribuir os dados de forma mais equilibrada, especialmente quando houver um número elevado de partições pequenas.
- `coalesce`: Use quando estiver reduzindo o número de partições para minimizar o shuffle e consolidar os dados, especialmente ao escrever dados para armazenamento.

### Evidências de repartição/Compactação

In [0]:
%fs ls /mnt/lhdw/gold/vendas_delta/

path,name,size,modificationTime
dbfs:/mnt/lhdw/gold/vendas_delta/dim_categoria/,dim_categoria/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/dim_cliente/,dim_cliente/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/dim_fabricante/,dim_fabricante/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/dim_geografia/,dim_geografia/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/dim_produto/,dim_produto/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/dim_segmento/,dim_segmento/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas/,fato_vendas/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas_repart/,fato_vendas_repart/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/geo_coalesce/,geo_coalesce/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/geo_coaslece/,geo_coaslece/,0,0


In [0]:
%fs ls /mnt/lhdw/gold/vendas_delta/fato_vendas_repart/

path,name,size,modificationTime
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas_repart/_delta_log/,_delta_log/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas_repart/part-00000-fb2ac92a-1433-4ada-9b02-696d3f4f04ce-c000.snappy.parquet,part-00000-fb2ac92a-1433-4ada-9b02-696d3f4f04ce-c000.snappy.parquet,1223034,1741985257000
dbfs:/mnt/lhdw/gold/vendas_delta/fato_vendas_repart/part-00001-dd8541cb-91a6-48fa-811a-0f95a7504b14-c000.snappy.parquet,part-00001-dd8541cb-91a6-48fa-811a-0f95a7504b14-c000.snappy.parquet,1221084,1741985258000


In [0]:
%fs ls /mnt/lhdw/gold/vendas_delta/geo_repart/

path,name,size,modificationTime
dbfs:/mnt/lhdw/gold/vendas_delta/geo_repart/_delta_log/,_delta_log/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/geo_repart/part-00000-6c84ffb7-5a12-4e76-8aef-496191b8a5c7-c000.snappy.parquet,part-00000-6c84ffb7-5a12-4e76-8aef-496191b8a5c7-c000.snappy.parquet,190375,1741985270000
dbfs:/mnt/lhdw/gold/vendas_delta/geo_repart/part-00001-6da892ed-299e-4a33-8131-2b1b36b25b3d-c000.snappy.parquet,part-00001-6da892ed-299e-4a33-8131-2b1b36b25b3d-c000.snappy.parquet,190532,1741985270000


In [0]:
%fs ls /mnt/lhdw/gold/vendas_delta/geo_regiao/

path,name,size,modificationTime
dbfs:/mnt/lhdw/gold/vendas_delta/geo_regiao/Regiao=Central/,Regiao=Central/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/geo_regiao/Regiao=East/,Regiao=East/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/geo_regiao/Regiao=West/,Regiao=West/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/geo_regiao/_delta_log/,_delta_log/,0,0


In [0]:
%fs ls /mnt/lhdw/gold/vendas_delta/geo_coalesce/

path,name,size,modificationTime
dbfs:/mnt/lhdw/gold/vendas_delta/geo_coalesce/_delta_log/,_delta_log/,0,0
dbfs:/mnt/lhdw/gold/vendas_delta/geo_coalesce/part-00000-ade5c30f-67d5-42f7-b24a-20ec3b13e386-c000.snappy.parquet,part-00000-ade5c30f-67d5-42f7-b24a-20ec3b13e386-c000.snappy.parquet,82254,1741985293000
dbfs:/mnt/lhdw/gold/vendas_delta/geo_coalesce/part-00001-c699f093-04b5-4e66-b46a-402f01ee3761-c000.snappy.parquet,part-00001-c699f093-04b5-4e66-b46a-402f01ee3761-c000.snappy.parquet,82739,1741985293000
dbfs:/mnt/lhdw/gold/vendas_delta/geo_coalesce/part-00002-f5762708-9d18-455b-966a-3d67a0bb58aa-c000.snappy.parquet,part-00002-f5762708-9d18-455b-966a-3d67a0bb58aa-c000.snappy.parquet,82612,1741985293000
dbfs:/mnt/lhdw/gold/vendas_delta/geo_coalesce/part-00003-7d6016d7-d967-4561-8400-a1d132d199a5-c000.snappy.parquet,part-00003-7d6016d7-d967-4561-8400-a1d132d199a5-c000.snappy.parquet,82562,1741985293000
dbfs:/mnt/lhdw/gold/vendas_delta/geo_coalesce/part-00004-dd0555ac-2e4f-4160-8c78-4293c221c972-c000.snappy.parquet,part-00004-dd0555ac-2e4f-4160-8c78-4293c221c972-c000.snappy.parquet,81950,1741985293000
